<a href="https://www.kaggle.com/code/xovotat/commit-classification-by-maintenance-activities?scriptVersionId=143398141" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Reproducing the multi-label commit classification method using transfer learning described in the [article](https://www.researchgate.net/profile/Mohamed-Wiem-Mkaouer-2/publication/348228961_Multi-label_Classification_of_Commit_Messages_using_Transfer_Learning/links/61eacfc2c5e3103375ae596d/Multi-label-Classification-of-Commit-Messages-using-Transfer-Learning.pdf) 

In [ ]:
import transformers

from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import json
import gc

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm

# Loading dataset

Using the given dataset from [here]( https://zenodo.org/record/3948448#.Y0kwAOxBw-Q)

In [ ]:
df = pd.read_csv('/kaggle/input/commit-labeling/train_2K.csv')
df.head()

## Data analyzing

In [ ]:
df['count'] = df['text'].apply(lambda x: len(x.split()))

In [ ]:
df.head()

## Counting the number of words in commit messages

In [ ]:
plt.figure(figsize= (8, 8))

sns.displot(df['count'])

plt.xlim(0, 80)

plt.xlabel('The num of words ', fontsize = 16)
plt.title("The Number of Words Distribution", fontsize = 18)
plt.show()

## Distribution by commit category

In [ ]:
corr_sum = sum([int(val) for val in df['Corrective']])
corr_sum

In [ ]:
perf_sum = sum([int(val) for val in df['Perfective']])
perf_sum

In [ ]:
adapt_sum = sum([int(val) for val in df['Adaptive']])
adapt_sum

In [ ]:
fig = plt.figure(figsize = (8, 5))

x = ['Corrective', 'Perfective', 'Adaptive']
y = [corr_sum, perf_sum, adapt_sum]

plt.xlabel('Categories', size = 15)

sns.barplot(x=x, y=y);
plt.show()

Extracting commit messages and corresponding labels 

In [ ]:
data_texts = df["text"].tolist()
data_labels = df[df.columns[2:5]].values

## Data split

Splitting of the data to *train* and *validation* groups using ratio 85%-15% proposed in the article

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.15, random_state=0)

# Model definition

In [ ]:
MODEL_NAME = 'distilbert-base-uncased'

### Encode with DistilBertTokenizer

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)

val_encodings = tokenizer(val_texts, truncation=True, padding=True)

## Turning labels and encodings into tf.Dataset object

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    list(train_labels)
))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    list(val_labels)
))

# Fine-tuning

Here the custom model is used in order to add a dense layer with sigmoid activation to the end of the model

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, num_labels):
        super(MyModel, self).__init__()
        self.distilbert = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
        self.fc = tf.keras.layers.Dense(num_labels, activation='sigmoid')

    def call(self, inputs, training=False):
        outputs = self.distilbert(inputs, training=training)
        pooled_output = outputs['logits']
        output = self.fc(pooled_output)
        return output


After several tests with different numbers of epochs, it was found out that the optimal number for teaching the model is 4 epochs (exactly the same parameter that was used in the article)

In [ ]:
BATCH_SIZE = 8
N_EPOCHS = 4

optimizerr = tf.keras.optimizers.Adam(learning_rate=2e-5)
losss = tf.keras.losses.BinaryCrossentropy()

## Model training

You do not need to train the model. Saved model can be used instead.

In [ ]:
model = MyModel(3)

model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=['accuracy'])

history = model.fit(train_dataset.batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE
         )

## Model saving

In [ ]:
model.save_weights('./checkpoints/my_checkpoint')

## Model loading

In [ ]:
saved_model = MyModel(3)
saved_model.load_weights('./checkpoints/my_checkpoint')

In [ ]:
saved_model.compile(optimizer=optimizerr,
                    loss=losss,
                    metrics=['accuracy'])

# Result evaluation

In [ ]:
history_eval = saved_model.evaluate(val_dataset.batch(BATCH_SIZE), return_dict=True, batch_size=BATCH_SIZE)

In [ ]:
loss_list = history.history['loss']

plt.plot(list(range(1, len(loss_list) + 1)), loss_list)
plt.xlabel('epochs', size = 15)
plt.ylabel('loss')
plt.xticks([1, 2, 3, 4])

The graph shows what error the model produces on test data during training at each step of four epochs

# Result validating

In [ ]:
def predict_proba(text_list):
    encodings = tokenizer(text_list, truncation=True, padding=True)
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings))) 
    return np.rint(model.predict(dataset)).astype(int)

In [ ]:
result = predict_proba(val_texts)

In [ ]:
result[:20]

In [ ]:
val_labels[:20]

In [ ]:
incorrect_elems = 0
for i in range(len(result)):
    for j in range(3):
        if result[i][j] != val_labels[i][j]:
            incorrect_elems += 1

Calculating ***hamming loss*** - percentage of incorrect labels

In [ ]:
hamming_loss = (incorrect_elems / (len(result) * 3))
hamming_loss

# Model testing on NNGen datasets 

**Test dataset**

In [ ]:
test_file = open('/kaggle/input/nngen-test/cleaned.test.msg', 'r').readlines()
valid_file = open('/kaggle/input/nngen-test/cleaned.valid.msg', 'r').readlines()

In [ ]:
test_texts = [elem.lower() for elem in test_file]
valid_texts = [elem.lower() for elem in valid_file]

In [ ]:
res_test = predict_proba(test_texts)

In [ ]:
corr_sum_test = sum(list(elem[0] for elem in res_test))
corr_sum_test

In [ ]:
perf_sum_test = sum(list(elem[1] for elem in res_test))
perf_sum_test

In [ ]:
adapt_sum_test = sum(list(elem[2] for elem in res_test))
adapt_sum_test

In [ ]:
fig = plt.figure(figsize = (8, 5))

x = ['Corrective', 'Perfective', 'Adaptive']
y = [corr_sum_test, perf_sum_test, adapt_sum_test]

plt.xlabel('Categories', size = 15)

sns.barplot(x=x, y=y);
plt.show()

**Validate dataset**

In [ ]:
res_valid = predict_proba(valid_texts)

In [ ]:
corr_sum_valid = sum(list(elem[0] for elem in res_test))
corr_sum_valid


In [ ]:
perf_sum_valid = sum(list(elem[1] for elem in res_test))
perf_sum_valid

In [ ]:
adapt_sum_valid = sum(list(elem[2] for elem in res_test))
adapt_sum_valid

In [ ]:
fig = plt.figure(figsize = (8, 5))

x = ['Corrective', 'Perfective', 'Adaptive']
y = [corr_sum_valid, perf_sum_valid, adapt_sum_valid]

plt.xlabel('Categories', size = 15)

sns.barplot(x=x, y=y);
plt.show()

Summation: the model showed the same result on the testing and validation sets